In [ ]:
import sqlite3
from py2neo import Graph, Node, Relationship

# 📌 Connexion à la base de données SQLite
db_path = r"C:\Users\JadS\Desktop\IUT\SAE_MigrationNoSQL\BD\bdd"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 📌 Connexion à Neo4j
neo4j_graph = Graph("bolt://localhost:7687", auth=("neo4j", "password7"))

# 📌 Suppression des nœuds et relations existants dans Neo4j (⚠️ Optionnel)
neo4j_graph.delete_all()

# 🔹 Fonction pour créer des nœuds et relations dans Neo4j
def creer_graphe_neo4j():
    print("🚀 Début de la création du graphe Neo4j...")

    tx = neo4j_graph.begin()  # Démarrer une transaction Neo4j

    # 📌 Création des départements
    cursor.execute("SELECT id_dep, nom_departement FROM Departement")
    departements = cursor.fetchall()
    print(f"📍 {len(departements)} départements trouvés.")

    departement_nodes = {}  # Stockage des nœuds pour éviter les requêtes répétées
    for id_dep, nom_departement in departements:
        node = Node("Departement", id_dep=id_dep, nom_departement=nom_departement)
        tx.create(node)
        departement_nodes[id_dep] = node
        print(f"✅ Département ajouté : {nom_departement} ({id_dep})")

    # 📌 Création des brigades et relations avec départements
    cursor.execute("SELECT id_brigade, id_dep, lib_brigade, type_brigade, perimetre FROM Brigade")
    brigades = cursor.fetchall()
    print(f"🚔 {len(brigades)} brigades trouvées.")

    brigade_nodes = {}  # Stockage des nœuds pour éviter les requêtes répétées
    for id_brigade, id_dep, lib_brigade, type_brigade, perimetre in brigades:
        node = Node("Brigade", id_brigade=id_brigade, lib_brigade=lib_brigade, type_brigade=type_brigade, perimetre=perimetre)
        tx.create(node)
        brigade_nodes[id_brigade] = node
        print(f"✅ Brigade ajoutée : {lib_brigade} ({id_brigade})")

        # Relation avec le département
        if id_dep in departement_nodes:
            tx.create(Relationship(node, "APPARTIENT_A", departement_nodes[id_dep]))
            print(f"🔗 Relation créée : {lib_brigade} APPARTIENT_A {departement_nodes[id_dep]['nom_departement']}")

    # 📌 Calcul du nombre total d'occurrences de chaque crime par département
    cursor.execute("SELECT id_crime, id_dep, SUM(nb_occurrences) FROM Enregistrer GROUP BY id_crime, id_dep")
    crime_occurrences_per_dep = {(row[0], row[1]): row[2] for row in cursor.fetchall()}

    # 📌 Création des crimes
    cursor.execute("SELECT code_index, lib_index FROM Crime")
    crimes = cursor.fetchall()
    print(f"⚖️ {len(crimes)} crimes trouvés.")

    crime_nodes = {}  # Stockage des nœuds pour éviter les requêtes répétées
    for code_index, lib_index in crimes:
        node = Node("Crime", code_index=code_index, lib_index=lib_index)
        tx.create(node)
        crime_nodes[code_index] = node
        print(f"✅ Crime ajouté : {lib_index} ({code_index})")

    # 📌 Création des relations "ENREGISTRE" par lots
    cursor.execute("SELECT id_enregistrement, id_brigade, id_crime, id_dep, annee, nb_occurrences FROM Enregistrer")
    enregistrements = cursor.fetchall()
    print(f"📊 {len(enregistrements)} enregistrements trouvés.")

    batch_size = 1000  # Taille du lot
    for i in range(0, len(enregistrements), batch_size):
        batch = enregistrements[i:i + batch_size]
        for id_enregistrement, id_brigade, id_crime, id_dep, annee, nb_occurrences in batch:
            brigade_node = brigade_nodes.get(id_brigade)
            crime_node = crime_nodes.get(id_crime)

            if brigade_node and crime_node:
                # Nombre total d'occurrences de ce crime dans ce département
                total_occurrences_dep = crime_occurrences_per_dep.get((id_crime, id_dep), 0)
                
                # Créer la relation entre la brigade et le crime
                relation = Relationship(
                    brigade_node, "ENREGISTRE", crime_node,
                    annee=annee,
                    nb_occurrences_brigade=nb_occurrences,  # Nombre spécifique à la brigade
                    nb_occurrences_departement=total_occurrences_dep  # Total dans le département
                )
                tx.create(relation)
                print(f"✅ Relation créée : {brigade_node['lib_brigade']} ENREGISTRE {crime_node['lib_index']} ({annee})")
                print(f"   🔹 {nb_occurrences} pour la brigade | 🔹 {total_occurrences_dep} pour le département")

        tx.commit()  # Valider les opérations par lot
        tx = neo4j_graph.begin()  # Démarrer une nouvelle transaction

    print("🎯 Création du graphe Neo4j terminée avec succès.")

# Appel de la fonction pour créer le graphe
creer_graphe_neo4j()

# 📌 Fermeture des connexions
conn.close()
print("✅ Graphe Neo4j créé avec succès et base SQLite fermée.")
